# Trabajo de Fin de Grado - Álvaro López García
## _Grado en Ingeniería Informática_
### Facultad de Ciencias - Universidad de Cantabria


In [1]:
# Celda para importar librerías, costantes, etc.
import pandas as pd
import tensorflow as tf

PATH = '/Volumes/TheVault/Documentos Mac/Documentos Universidad/4o Curso/2o Cuatrimestre/Trabajo de Fin de Grado/maestro-v3.0.0/maestro-v3.0.0.csv'

Creamos el dataframe

In [2]:
def get_dataframe(path=PATH):
    df = pd.read_csv(path)
    return df

In [3]:
df = get_dataframe()
df.head()

,canonical_composer,canonical_title,split,year,midi_filename,audio_filename,duration
0,Alban Berg,Sonata Op. 1,train,2018,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...,698.661160
1,Alban Berg,Sonata Op. 1,train,2008,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...,759.518471
2,Alban Berg,Sonata Op. 1,train,2017,2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-...,2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-...,464.649433
3,Alexander Scriabin,"24 Preludes Op. 11, No. 13-24",train,2004,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MI...,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MI...,872.640588
4,Alexander Scriabin,"3 Etudes, Op. 65",validation,2006,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MI...,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MI...,397.857508


Creación del modelo

In [19]:
# convolutional operation parameters
n_filters = 32 
filter_width = 2
dilation_rates = [2**i for i in range(11)] * 3
inp_shape=(4096, 1)

# define an input history series and pass it through a stack of dilated causal convolution blocks. 
Input_seq = tf.keras.layers.Input(shape=inp_shape, dtype='float32')
x = Input_seq

skips = []
for dilation_rate in dilation_rates:

    # preprocessing - equivalent to time-distributed dense
    x = tf.keras.layers.Conv1D(16, 1, padding='same', activation='relu')(x) 
    
    # filter convolution
    x_f = tf.keras.layers.Conv1D(filters=n_filters,
                 kernel_size=filter_width, 
                 padding='same',
                 dilation_rate=dilation_rate)(x)

    # gating convolution
    x_g = tf.keras.layers.Conv1D(filters=n_filters,
                 kernel_size=filter_width, 
                 padding='same',
                 dilation_rate=dilation_rate)(x)

    # multiply filter and gating branches
    z = tf.keras.layers.Multiply()([tf.keras.layers.Activation('tanh')(x_f),
                    tf.keras.layers.Activation('sigmoid')(x_g)])

    # postprocessing - equivalent to time-distributed dense
    z = tf.keras.layers.Conv1D(16, 1, padding='same', activation='relu')(z)

    # residual connection
    x = tf.keras.layers.Add()([x, z])    
    
    # collect skip connections
    skips.append(z)

# add all skip connection outputs 
out = tf.keras.layers.Activation('relu')(tf.keras.layers.Add()(skips))
model = tf.keras.models.Model(Input_seq, out)
model.compile(tf.keras.optimizers.Adam(), loss='mean_absolute_error')
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 4096, 1)]    0           []                               
                                                                                                  
 conv1d_1060 (Conv1D)           (None, 4096, 16)     32          ['input_11[0][0]']               
                                                                                                  
 conv1d_1061 (Conv1D)           (None, 4096, 32)     1056        ['conv1d_1060[0][0]']            
                                                                                                  
 conv1d_1062 (Conv1D)           (None, 4096, 32)     1056        ['conv1d_1060[0][0]']            
                                                                                            